## Writing Functions

The goal of this exercise is to replicate the process from the slides, but with multivariate data. You should think of this as a practise of two things: 

1. Performing linear algebra in R. This is straight forward, but takes some practise. Especially worth noting how to use vectors and scalars together, as you see how to modify the functions we originally wrote for scalars, such that they work for vectors.

2. Writing small functions and testing them. Make sure each part works before trying to combine them into a whole!

In [37]:
library(tidyverse)

In [38]:
# Write a function that generates data for regressions, 

# y should be generated according to: 
# y <- X %*% beta + eps
# X should be several columns of independent random normals (aka a multivariate random 
# normal matrix with diagonal covariance)
# eps should be such that:
# eps <- rnorm(1, 0, sd)

# beta should be a vector
# params should be a DATAFRAME/TIBBLE with two columns: "mean" and "sd" to generate the features (X) 
# sd should be a scalar: the standard deviation of the normally distributed error term in the DGP

# hint: use cbind to bind vectors into the columns of a matrix or the library MASS with the function mvrnorm


# generate_data <- function(N, beta, params, sd) { 
#     list(X=X, y=y)
# }

In [39]:
generate_data <- function(N, beta, sd) {
    x <- rnorm(N, 0, 1)
    eps <- rnorm(N, 0, sd)
    y <- beta*x + eps
    list(x = x, y = y)
}

data <- generate_data(10, 3, 3)
data

$x
 [1]  0.8278299  0.5046478  1.4635237  0.3779689 -0.9383846  1.9266595
 [7]  0.6505254  1.0794469  2.0227641 -0.1805636

$y
 [1] -1.63728617 -2.51448523  3.31752119  3.86850973 -2.18834425  2.32533099
 [7]  4.62794280 -0.04560812  9.20390392  4.57329654

In [40]:
# OLS regression

calc_coef <- function(y,x) cov(x,y) / var(x)

calc_se <- function(y, x, coef) {
    n <- length(y)
    eps <- y - x*coef
    e_sd <- mean(eps^2)
    se <- sqrt(e_sd / (n*var(x)))
    se
}

run_regression <- function(y, x) {
    coef <- calc_coef(y, x)
    se <- calc_se(y, x, coef)
    list(coef=coef, se=se)
}

In [41]:
run_regression(data$y, data$x)

$coef
[1] 1.898161

$se
[1] 1.110152

In [42]:
# Write other helper functions, as in the slides, and eventually a function "avg_simulations" that takes
# the same parameters as generate_data, plus an "M" parameter that controls how many simulations are run

In [43]:
eval_model <- function(coef, se, beta, conf = 1.96) {
    up <- coef + se*conf
    down <- coef - se*conf
    beta > down & beta < up
}

simulate <- function(N, beta, sd) {
    d <- generate_data(N, beta, sd)
    m <- run_regression(d$y, d$x)
    eval_model(m$coef, m$se, beta)
}

avg_simulations <- function(M, N, beta, sd) {
    inside <- sapply(1:M, function (x) {
        simulate(N, beta, sd)
    })
    sum(inside) / M
}

In [44]:
simulate(100, .5, .5)

[1] TRUE

In [47]:
1:10 #vector of integers

[1]  1  2  3  4  5  6  7  8  9 10

In [56]:
applier <- function(x) {
    # x is integer
    print(x)
    x**2
}

## different return
sapply(1:10, applier) ## returns vector
# lapply(1:10, applier) ## returns list

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10


[1]   1   4   9  16  25  36  49  64  81 100

In [63]:
applier <- function(N, beta, sd) {
    # x is integer
    simulate(N, beta, sd)
}

sapply(1:10, function (x) applier(100, .5, .5)) ## lambda fucntion

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [73]:
M <- 10
# output <- rep(TRUE, M)
output <- rep(0, M)

## automatically converts booleans if it was an integeres vector before
for (i in 1:M) {
    output[i] <- simulate(100, .5, .5)
}

output

[1] 1 1 1 1 0 1 1 1 1 1

In [79]:
run <- function(M, N, beta, sd) {
    res <- sapply(1:M, function (x) simulate(N, beta, sd))
    sum(res) / M
}
                  
run(1000, 100, .5, .5)

[1] 0.935

## Tests

Make sure the following assertions pass:

In [45]:
a <- avg_simulations(1000, 
                20, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(round(a, 1) == .9)

ERROR: Error in avg_simulations(1000, 20, c(1, 2, 1), data.frame(list(mean = c(0, : unused argument (0.5)


In [46]:
a <- avg_simulations(1000, 
                500, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(a > .92)

ERROR: Error in avg_simulations(1000, 500, c(1, 2, 1), data.frame(list(mean = c(0, : unused argument (0.5)
